<a href="https://colab.research.google.com/github/xHexlabx/SuperAI_SS4_Recap/blob/main/SuperAI_SS4_Level_1/Hack_1_Image_Search/Image_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SuperAI Season 4 - Level 1 Hackathon - Image Search

# Explore Datasets

In [1]:
import cv2
import numpy as np

In [2]:
def refine_image (image) :

    width = 336
    height = 336

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image , (width , height))
    image = np.array(image)[: , : , 0 : 3]

    return image

In [4]:
refined_image_path = './datasets/queries/refined_queries'

for i in range(22) :

    image = cv2.imread(f'./datasets/queries/queries/{i}.jpg')

    resized_image = refine_image(image)

    with open(f'{refined_image_path}/{i}.npy' , 'wb') as file :

        np.save(file , resized_image)

# CLIP VisionModel For Encode Image

In [5]:
!nvidia-smi

'DOSKEY' is not recognized as an internal or external command,
operable program or batch file.
'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
from PIL import Image
import requests
from transformers import AutoProcessor, CLIPModel
import torch

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CLIPModel.from_pretrained('openai/clip-vit-large-patch14')
processor = AutoProcessor.from_pretrained('openai/clip-vit-large-patch14')

config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

c:\anaconda\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\teehe\.cache\huggingface\hub\models--openai--clip-vit-large-patch14. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [8]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [9]:
inputs = processor(images = image, return_tensors = "pt")
image_features = model.get_image_features(**inputs)

In [10]:
image_features.shape

torch.Size([1, 768])

# Solution-CLIP Zeroshot For Image Similarity Search

## Cosine Similarity

In [11]:
import numpy as np
from torch import nn

def cosine_similarity (A , B) :

    A = np.array(A)
    B = np.array(B)

    norm_A = np.linalg.norm(A)

    similarity_matrix = (A @ B.T).astype('float32') / norm_A

    for i in range(B.shape[0]) :

        norm_B = np.linalg.norm(B[i])

        similarity_matrix[0][i] /= norm_B

    similarity_matrix = similarity_matrix.reshape(-1)

    # softmax
    # similarity_matrix = np.exp(similarity_matrix)/sum(np.exp(similarity_matrix))

    target_class = similarity_matrix.argmax(axis = 0)
    maximum_similarity = similarity_matrix[target_class]

    return maximum_similarity , target_class

In [12]:
print(cosine_similarity(np.array([[1 , 2  ,3]]) , np.array([[1 , 2 , 3] , [3 , 4 , 5]])))

(0.99999994, 0)


## Define Model CLIP

In [13]:
from PIL import Image
import requests
from transformers import AutoProcessor , CLIPModel

## openai/clip-vit-large-patch14

In [14]:
import torch

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CLIPModel.from_pretrained('openai/clip-vit-large-patch14').to(device)
processor = AutoProcessor.from_pretrained('openai/clip-vit-large-patch14')

## Read queries folder image and store to numpy array

In [16]:
import cv2
import os
import numpy as np

In [17]:
refined_queries_path = './datasets/queries/refined_queries'
images = []

for file_name in os.listdir(refined_queries_path) :

    with open(f'{refined_queries_path}/{file_name}' , 'rb') as file :

        image = np.load(file)

    images.append(image)

inputs = processor(images = images , return_tensors = 'pt').to(device)
image_features = model.get_image_features(** inputs)

In [18]:
image_features = image_features.cpu().detach().numpy()

In [19]:
with open('./database/queries_matrix.npy' , 'wb') as file :

    np.save(file , image_features)


## Read test folder images to inference

In [20]:
from tqdm import tqdm
import pandas as pd

In [21]:
with open('./database/queries_matrix.npy' , 'rb') as file :

    queries_matrix = np.load(file)

In [22]:
queries_matrix.shape

(22, 768)

In [23]:
submissions = {
    'img_file' : [] ,
    'class' : []
}

In [24]:
threshold = 0

In [25]:
test_path = './datasets/test/images'

for file_name in tqdm(os.listdir(test_path)) :

    image = cv2.imread(f'{test_path}/{file_name}')
    image = refine_image(image)

    inputs = processor(images = image , return_tensors = 'pt').to(device)
    image_features = model.get_image_features(** inputs)

    maximum_similarity , target_class = cosine_similarity(image_features.cpu().detach().numpy() , queries_matrix )

    threshold += maximum_similarity

100%|██████████| 1120/1120 [23:14<00:00,  1.25s/it]


In [26]:
threshold /= 1120
threshold

0.8226966738700867

In [27]:
test_path = './datasets/test/images'

for file_name in tqdm(os.listdir(test_path)) :

    image = cv2.imread(f'{test_path}/{file_name}')
    image = refine_image(image)

    inputs = processor(images = image , return_tensors = 'pt').to(device)
    image_features = model.get_image_features(** inputs)

    maximum_similarity , target_class = cosine_similarity(image_features.cpu().detach().numpy() , queries_matrix )

    if maximum_similarity < threshold :

        target_class = 22

    submissions['img_file'].append(file_name)
    submissions['class'].append(target_class)

100%|██████████| 1120/1120 [26:51<00:00,  1.44s/it]


In [28]:
submissions = pd.DataFrame.from_dict(submissions)
submissions_path = './submissions/submissions.csv'

with open(submissions_path, 'w') as csv_file:

    submissions.to_csv(path_or_buf = csv_file , index = False)

In [29]:
submissions.groupby('class').count()

,img_file
class,
0,4
1,42
2,68
3,8
4,1
5,6
6,20
7,3
10,10
